In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera,\
                      RadioMapSolver, PathSolver,transform_mesh
from sionna.rt.utils import r_hat, subcarrier_frequencies

In [14]:
scene = load_scene("Hong Hum_256.xml",
                   merge_shapes=False)


# Configure a transmitter that is located at the front of "car_2"
scene.add(Transmitter("tx", position=[6,18,1.5], orientation=[np.pi,0,0]))
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, pattern="tr38901", polarization="V")
scene.rx_array = scene.tx_array

# Create radio map solver
rm_solver = RadioMapSolver()
                # rm_show_color_bar=True,
rm = rm_solver(scene=scene,
                   samples_per_tx=20**6,
                   refraction=True,
                   max_depth=10,
                   cell_size=[2,2])
cam =  Camera(position=[-14,35,450], look_at=[-14,35,0])
# scene.preview(radio_map=rm,
#                 rm_vmax=-40, rm_vmin=-150)
scene.render_to_file(camera=cam,  filename="HH.png", radio_map=rm,
                     resolution=(256,256),
                num_samples=1024,
                rm_vmin=-140)


Bitmap[
  pixel_format = rgba,
  component_format = uint8,
  size = [256, 256],
  srgb_gamma = 1,
  struct = Struct<4>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
    uint8 A; // @3, normalized, alpha
  ],
  data = [ 256 KiB of image data ]
]

In [ ]:
for i in range(num_displacements+1):

    # Compute and render a coverage map at 0.5m above the ground
    rm = rm_solver(scene=scene,
                   samples_per_tx=20**6,
                   refraction=True,
                   max_depth=10,
                   center=[0,0,0.5],
                   orientation=[0,0,0],
                   size=[186,121],
                   cell_size=[2,2])
    # scene.render(camera=cam,  radio_map=rm,
    #              num_samples=512, resolution=(256,256),
    #              rm_show_color_bar=True,
    #              rm_vmax=-40, rm_vmin=-150)
    scene.preview(radio_map=rm,
                 resolution=(256,256),
                 rm_vmax=-40, rm_vmin=-150)
    # scene.render_to_file(camera=cam, filename = f"{i}.png", radio_map=rm,
    #              num_samples=512,resolution=(256,256),
    #              rm_vmax=-40, rm_vmin=-150)
    # # Move TX to next position
    # scene.get("tx").position -= displacement_vec

    # Move cars driving in -x direction
    for j in range(1,6):
        scene.get(f"car_{j}").position -= displacement_vec

    # Move cars driving in x direction
    for j in range(6,9):
        scene.get(f"car_{j}").position += displacement_vec

In [ ]:
scene = load_scene(sionna.rt.scene.simple_street_canyon_with_cars,
                   merge_shapes=False)
cam =  Camera(position=[50,0,130], look_at=[10,0,0])


# Configure a transmitter that is located at the front of "car_2"
scene.add(Transmitter("tx", position=[22.7, 5.6, 25], orientation=[np.pi,0,0]))
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, pattern="tr38901", polarization="V")
scene.rx_array = scene.tx_array

# Create radio map solver
rm_solver = RadioMapSolver()

# Move cars along straight lines for a couple of steps
displacement_vec = [10, 0, 0]
num_displacements = 2

car_materal = sionna.rt.ITURadioMaterial("car-material","metal",thickness = 0.01,color = (0.8, 0.1, 0.1))
num_cars = 1
cars = [sionna.rt.SceneObject(fname = sionna.rt.scene.low_poly_car, name = f"car{i}", radio_material=car_materal) for i in range(num_cars)]
scene.edit(add=cars)
cars[0].position  = scene.get(f"car_1").position + [2, 2, 0]

cars[0].orientation = [0, 0, 0]
for i in range(num_displacements+1):

    # Compute and render a coverage map at 0.5m above the ground
    rm = rm_solver(scene=scene,
                   samples_per_tx=20**6,
                   refraction=True,
                   max_depth=10,
                   center=[0,0,0.5],
                   orientation=[0,0,0],
                   size=[186,121],
                   cell_size=[2,2])
    # scene.render(camera=cam,  radio_map=rm,
    #              num_samples=512, resolution=(256,256),
    #              rm_show_color_bar=True,
    #              rm_vmax=-40, rm_vmin=-150)
    scene.preview(radio_map=rm,
                 resolution=(256,256),
                 rm_vmax=-40, rm_vmin=-150)
    # scene.render_to_file(camera=cam, filename = f"{i}.png", radio_map=rm,
    #              num_samples=512,resolution=(256,256),
    #              rm_vmax=-40, rm_vmin=-150)
    # # Move TX to next position
    # scene.get("tx").position -= displacement_vec

    # Move cars driving in -x direction
    for j in range(1,6):
        scene.get(f"car_{j}").position -= displacement_vec

    # Move cars driving in x direction
    for j in range(6,9):
        scene.get(f"car_{j}").position += displacement_vec

In [ ]:
# Visualize path gain
rm.show(metric="path_gain");

# Visualize received signal strength (RSS)
rm.show(metric="rss");

# Visulaize SINR
rm.show(metric="sinr");

In [ ]:
# Move the car 10m along the y-axis
car_2.position += [0, 10, 0]

# And rotate it by 90 degree around the z-axis
car_2.orientation = [np.pi/2, 0, 0]